In [5]:
import numpy as np
import matplotlib.pyplot as plt
import importlib
import sys

import astropy
import astropy.units as u
import astropy.constants as c
from astropy.cosmology import default_cosmology

In [2]:
# Plot Styling
plt.style.use('dark_background')
plt.rcParams.update({
    'font.family':'serif', 'mathtext.fontset':'dejavuserif',
    'axes.grid':True, 'grid.linestyle': ':', 'grid.alpha': 0.5,
    'xtick.direction':'in', 'xtick.minor.visible': True, 'xtick.top':True,
    'ytick.direction':'in', 'ytick.minor.visible': True, 'ytick.right':True,
    'figure.figsize': [9, 6], 'axes.titlesize':30, 'legend.fontsize': 15, 'legend.title_fontsize': 20,
    'axes.labelsize':25, 'xtick.labelsize':15, 'ytick.labelsize':15
})

In [6]:
sys.path.append('/global/homes/c/cpopik/Capybara')
from Models import SHMRs, HaloModels, HODs, SMFs

fmoddict = {
    'SHMR': {
        'name':'Kravstov2014', 
        'spefs': {
            'sample':'M200c'}},
    'HMF': {
        'name':'pyccl', 
        'spefs': {
            'mfunc':'Tinker08', 
            'mdef':'200c', 
            'hbias':'Tinker10'}},
    'SMF': {
        'name': 'BOSSDR10', 
        'spefs': {
            'galaxy':'CMASS', 
            'group':'portsmouth', 
            'template':'passive', 
            'IMF':'Kroupa'}},
    'HOD': {
        'name':'Kou2023', 
        'spefs': {
            'sample':'M*>10.8'}},}

hod = getattr(HODs, fmoddict['HOD']['name'])(fmoddict['HOD']['spefs'])
shmr = getattr(SHMRs, fmoddict['SHMR']['name'])(fmoddict['SHMR']['spefs'])
smf = getattr(SMFs, fmoddict['SMF']['name'])(fmoddict['SMF']['spefs'])
hmod = getattr(HaloModels, fmoddict['HMF']['name'])(fmoddict['HMF']['spefs'])

In [7]:
params = {
    "hh": {
        "value": 0.7},
    "Omega_L": {
        "value": 0.75},
    "Omega_m": {
        "value": 0.25},
    "Omega_b": {
        "value": 0.044},
    "T_CMB": {
        "value": 2.725},
    "XH": {
        "value": 0.76},        # hydrogen fraction
    "v_rms": {
        "value": 1.06e-3}   # v_rms/c, used for kSZ
}

cpars = {k: v["value"] for k, v in params.items() if isinstance(v, dict) and "value" in v}  

In [27]:
# Radial bins are chosen somewhat arbitrarily here
rs = np.logspace(-1.5, 1.5, 100)

# Add limits and bins for redshift and mass
zbins = np.linspace(0.4, 0.8, 21)
logmhalobins = np.linspace(12, 14, 51)

# Get the SMF and distributions, and the z and mstar arrays
hmf_smf = smf.SMF_to_HMF(logmhalobins, shmr.SHMR, zbins=zbins, **cpars)
ndist = smf.Ndist_halo(logmhalobins, shmr.SHMR, zbins=zbins, **cpars)
zs, logmhalos = smf.z, smf.logmhalo

In [9]:
cosmology = astropy.cosmology.LambdaCDM(H0=cpars["hh"]*100, Tcmb0=cpars['T_CMB'], Om0=cpars["Omega_m"], Ode0=cpars["Omega_L"], Ob0=cpars["Omega_b"])

Hs = cosmology.H(zs).to(u.km/u.s/u.Mpc).value
rhocs = cosmology.critical_density(zs).to(u.Msun/u.Mpc**3).value
dAs = cosmology.angular_diameter_distance(zs).value
r200cs = (10**logmhalos/(4/3*np.pi*200*rhocs[:, None]))**(1/3)

# We'll also need average angular diameter distance for the projection later
zave = np.sum(smf.N()*smf.z[:, None])/smf.N().sum()
dA_ave = cosmology.angular_diameter_distance(zave).value

In [10]:
from Models import FFTs

fft = FFTs.mcfit_package(rs)
ks = fft.ks

In [28]:
hmf_hmod = hmod.HMF(zs, logmhalos, **cpars)
bhs = hmod.bh(zs, logmhalos, **cpars)
Plins = hmod.Plin(ks, zs, **cpars)

In [14]:
from Models import Profiles, Data, Dust

tSZdict = {
    'data': {
        'name':'Schaan2021', 
         'spefs': {
             'freq': '150', 
             'sample':'cmass'}},
    'onehalo': {
        'name':'Amodeo2021', 
         'spefs': {
             'model':'GNFW'}},
    'twohalo': {
        'name':'Amodeo2021', 
         'spefs': {
             'model':'GNFW'}},
    'dust': {
        'name':'Amodeo2021',
        'spefs': {
            'model':'ACT+Hershel'}}}

# Define a pressure and dust profile
Pth1h = getattr(Profiles, tSZdict['onehalo']['name'])(tSZdict['onehalo']['spefs'])
Pth2h = getattr(Profiles, tSZdict['twohalo']['name'])(tSZdict['twohalo']['spefs'])
tSZmeas = getattr(Data, tSZdict['data']['name'])(tSZdict['data']['spefs'])
tSZdust = getattr(Dust, tSZdict['dust']['name'])(tSZdict['dust']['spefs'])

In [15]:
kSZdict = {
    'data': {
        'name':'Schaan2021', 
         'spefs': {
             'freq': '150', 
             'sample':'cmass'}},
    'onehalo': {
        'name':'Amodeo2021', 
        'spefs': {
            'model':'GNFW'}},
    'twohalo': {
        'name':'Amodeo2021', 
        'spefs': {
            'model':'GNFW'}},}

rho1h = getattr(Profiles, kSZdict['onehalo']['name'])(kSZdict['onehalo']['spefs'])
rho2h = getattr(Profiles, kSZdict['twohalo']['name'])(kSZdict['twohalo']['spefs'])
kSZmeas = getattr(Data, kSZdict['data']['name'])(kSZdict['data']['spefs'])

In [16]:
Pth = lambda params={}: Pth1h.Pth1h(rs, zs, logmhalos, rhocs, r200cs, **cpars)(params) + Pth2h.Pth2h(rs, zs, logmhalos, rhocs, r200cs, Plins, bhs, HMFs,fft.FFT3D, fft.IFFT3D, ks, **cpars)(params)[..., None]

rho = lambda params={}: rho1h.rho1h(rs, zs, logmhalos, rhocs, r200cs, **cpars)(params) + rho2h.rho2h(rs, zs, logmhalos, rhocs, r200cs, Plins, bhs, HMFs, fft.FFT3D, fft.IFFT3D, ks, **cpars)(params)[..., None]

In [ ]:
from Models import Spectra
importlib.reload(Spectra)

Nc, Ns = hod.Nc(logmhalos), hod.Ns(logmhalos)
uck, usk = hod.uck(), hod.usk(rs, r200cs, fft.FFT3D)

cross = Spectra.HODweighting(Nc, Ns, uck, usk, logmhalos, hmf_hmod, fft.FFT3D, fft.FFT1D, **cpars)

cross(Pth())

array([[1.06749492e-11, 1.09505523e-11, 1.12291237e-11, ...,
        1.57251684e-11, 1.60427428e-11, 1.63619874e-11],
       [1.11105986e-11, 1.13955644e-11, 1.16834220e-11, ...,
        1.63026588e-11, 1.66269196e-11, 1.69526056e-11],
       [1.05840065e-11, 1.08570135e-11, 1.11329397e-11, ...,
        1.55830815e-11, 1.58971693e-11, 1.62128750e-11],
       ...,
       [4.70784156e-16, 4.82635142e-16, 4.94805841e-16, ...,
        7.18143167e-16, 7.35889292e-16, 7.54004601e-16],
       [3.71344986e-16, 3.80676797e-16, 3.90262132e-16, ...,
        5.66406534e-16, 5.80419738e-16, 5.94726760e-16],
       [2.78532012e-16, 2.85496606e-16, 2.92653838e-16, ...,
        4.24674147e-16, 4.35211009e-16, 4.45973330e-16]])

In [74]:
cross(Pth())

TypeError: 'numpy.ndarray' object is not callable

In [ ]:
cpars = {"hh": 0.7, "Omega_L":0.7, "Omega_m":0.3, "Omega_b": 0.044, "XH": 0.76, "T_CMB":2.725}
logmshalo = np.linspace(10, 14, 50)
zs = np.linspace(0.4, 1.1, 10)
rs = np.logspace(-1.5, 1.5, 100)

cosmology = astropy.cosmology.LambdaCDM(H0=cpars["hh"]*100, Tcmb0=2.726, Om0=cpars["Omega_m"], Ode0=cpars["Omega_L"], Ob0=cpars["Omega_b"])
rhoc_func = lambda z: cosmology.critical_density(z).to(u.Msun/u.Mpc**3).value
r200c_func = lambda z, logm200c: (10**logm200c/(4/3*np.pi*200*rhoc_func(z[:, None])))**(1/3)
H_func = lambda z: cosmology.H(z).to(u.km/u.s/u.Mpc).value
chi_func = lambda z: cosmology.comoving_distance(z).value

rhocs = rhoc_func(zs)
rs200c = r200c_func(zs, logmshalo)
Hs = H_func(zs)
chis = chi_func(zs)

In [ ]:
import Models.FFTs as FFTs
ks, FFT_func = FFTs.mcfit_package(rs).FFT3D()
rs_rev, IFFT_func = FFTs.mcfit_package(rs).IFFT3D()
rs_rev, IFFT1D_func = FFTs.mcfit_package(rs).IFFT1D()
rs_rev, IFFT2D_func = FFTs.mcfit_package(rs).IFFT2D()


import Capybara.Models.HaloModels as HaloModels
hmfccl = HaloModels.pyccl({'mdef':'200c', 'mfunc':'Tinker08', 'hbias':'Tinker10'})
Pklin = hmfccl.Plin(ks, zs, **cpars)
bias = hmfccl.bh(zs, logmshalo, **cpars)
hmf = hmfccl.HMF(zs, logmshalo, **cpars)

import Models.HODs as HODs
importlib.reload(HODs)

Kou = HODs.Kou2023({'sample':'M*>10.8'})

import Models.Profiles as Profiles
importlib.reload(Profiles)

AmodeoPth_1h = Profiles.Amodeo2021({'model':'GNFW'}).Pth1h(rs, zs, logmshalo, rhocs, rs200c, **cpars)

In [ ]:
np.trapz(hmf, logmshalo)

In [ ]:
import Models.Spectra as Spectra
importlib.reload(Spectra)

Nc = Kou.Nc(logmshalo)()
Ns = Kou.Ns(logmshalo)()
uck = Kou.uck()
usk = Kou.usk(rs, rs200c, FFT_func)()
y_k = Spectra.y_k(FFT_func, **cpars)(AmodeoPth_1h())

Pgy = Spectra.Pgy1h(logmshalo, hmf, y_k, Hs, Nc, Ns, uck, usk)

zdist = scipy.stats.norm.pdf(zs, loc=0.5, scale=0.1) 
ells = np.geomspace(50, 10000, 100)
C_ells = Spectra.limber(ks, zs, Spectra.W_g(zdist), Spectra.W_y(zs), Hs, chis, ells)

test1 = Spectra.HODweighting(zs, logmshalo, hmf, FFT_func, IFFT1D_func,zdist)(AmodeoPth_1h(), H_g=Spectra.H_g(Nc, Ns, uck, usk, logmshalo, hmf))
test2 = Spectra.HODweighting(zs, logmshalo, hmf, FFT_func, IFFT1D_func, zdist,H_g=Spectra.H_g(Nc, Ns, uck, usk, logmshalo, hmf))(AmodeoPth_1h())

In [ ]:
plt.plot(rs, zdist)

In [ ]:
np.average(zs, weights=zdist)

In [ ]:
zs[1]-zs[0]

In [ ]:
np.trapz(zdist)/(zs[1]-zs[0])

In [ ]:
AmodeoPth_1h()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 5), layout='constrained')

axs[0].loglog(ks, Pgy[:,4])

axs[1].loglog(ells, C_ells(Pgy))

axs[2].loglog(ells, ells*(ells+1)*C_ells(Pgy)/2/np.pi)

axs[0].set(xlabel=r'$k$', ylabel=r'$P_{gy}$'); axs[0].legend()
axs[1].set(xlabel=r'$\ell$', ylabel=r'$C_\ell$'); axs[1].legend()
axs[2].set(xlabel=r'$\ell$', ylabel=r'$D_\ell$'); axs[2].legend()

plt.show()

In [ ]:
plt.loglog(ks, C_ells(Spectra.cross1h(logmshalo, hmf, Hg(), Hy())))

In [ ]:
scipy.stats.norm.pdf(zs, loc=0.5, scale=0.1) 

In [ ]:
(FFTs.mcfit_package(np.logspace(-1, 1)).FFT3D()[0][:, None]*chis).min()

In [ ]:
lmin, lmax, nell = 100, 10000, 20

ells_from_ks = ks[:, None]*chis
ell_min = np.max([lmin, (ells_from_ks.min()//10 + 1)*10])
ell_max = np.min([lmax, (ells_from_ks.max() // 1e3)*1e3])
ells = np.geomspace(ell_min, ell_max, nell)
ks_from_ells = ells[:, None]/chis

tupletest = np.stack((ks_from_ells, zs*np.ones(ks_from_ells.shape)), axis=-1)

intp = RegularGridInterpolator((ks, zs), thing, bounds_error=False, fill_value=0)

intp(tupletest).shape

In [ ]:
np.trapz(intp(tupletest), axis=1)